<a href="https://colab.research.google.com/github/almutareb/InnovationPathfinderAI/blob/main/Copy_of_mixtral_react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain_community
!pip install -qU langchain
!pip install -qU google-search-results
!pip install -qU langchainhub
!pip install -qU text_generation
!pip install -qU arxiv
!pip install -qU wikipedia
!pip install -qU gradio==3.48.0
!pip install -qU youtube_search
!pip install -qU sentence_transformers
!pip install -qU hromadb

In [ ]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
#os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "arxiv_ollama_agent"

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wikipedia.run("large language model")

In [ ]:
websearch = GoogleSearchAPIWrapper()

def top5_results(query):
  return websearch.results(query, 5)

google_search = Tool(
    name="google_search",
    description="Search Google for recent results.",
    #func=top5_results,
    func=websearch.run,
)

In [ ]:
google_search.run("large language model")

In [ ]:
wikipedia.args

In [ ]:
# HF libraries
from langchain.llms import HuggingFaceHub

# Load the model from the Hugging Face Hub
model_id = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={
    "temperature":0.1,
    "max_new_tokens":1024,
    "repetition_penalty":1.2,
    "return_full_text":False
    })

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain.tools.render import render_text_description
from langchain.tools.retriever import create_retriever_tool
from langchain.retrievers import ArxivRetriever
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools import YouTubeSearchTool

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

arxiv_retriever = ArxivRetriever()

arxiv_search = create_retriever_tool(
    arxiv_retriever,
    "arxiv_database",
    "Search arxiv database for scientific research papers and studies",
)

youtube_search = YouTubeSearchTool()

tools = [arxiv_search, wikipedia, google_search]

#prompt = hub.pull("hwchase17/react")
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
chat_model_with_stop = model_id.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
#    | model_id
    | ReActJsonSingleInputOutputParser()
)

#agent = create_react_agent(model_id, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,       # cap number of iterations
    #max_execution_time=60,  # timout at 60 sec
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    )

def stream_output(query):
  for chunk in agent_executor.stream({"input": query}):
      # Agent Action
      if "actions" in chunk:
          for action in chunk["actions"]:
              print(
                  f"Calling Tool ```{action.tool}``` with input ```{action.tool_input}```"
              )
      # Observation
      elif "steps" in chunk:
          for step in chunk["steps"]:
              print(f"Got result: ```{step.observation}```")

# Chat memory not working yet
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_message_key="input",
    history_messages_key="chat_history",
)

In [ ]:
stream_output("what is corrective retrieval augmeneted generation")

In [ ]:
## Youtube search tool, not used yet
import ast
def you_four(query):
  fquery = query+',4'
  videos_str = youtube_search.run(fquery)
#  video_list.replace('watch?v=','embed/')
#  video_list = [word.replace('watch?v=','embed/') for word in video_list]
  video_list = convert_urls(videos_str)

  return video_list

def convert_urls(urls):
  # Convert the string representation of the list into an actual list
  urls = ast.literal_eval(urls)
  #urls = [ for url in urls]
  iframes = []
  for url in urls:
    embed_url = url.replace('watch?v=','embed/')
    iframe = f'<iframe width="560" height="315" src="{embed_url}" frameborder="0" allowfullscreen></iframe>'
    iframes.append(iframe)
  return iframes

In [ ]:
list_d=you_four("air taxi")
list_d

In [ ]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

In [ ]:
return_txt= agent_executor.invoke(
    {
         "input": "how could a concept for an airtaxi fleet management look like?",
    }
)

In [ ]:
agent_executor.invoke(
    {
         "input": "What's the latest paper on corrective retrieval augmeneted generation?"
    }
)

In [ ]:

import gradio as gr
def add_text(history, text):
    history = history + [(text, None)]
    return history, ""

def bot(history):
    response = infer(history[-1][0], history)
    history[-1][1] = response['output']
    return history

def infer(question, history):
    query =  question
    result = agent_executor.invoke(
        {
            "input": question,
        }
    )
    return result

def you_frame(question):
  iframes=you_four(question)
  return '\n'.join(iframes)

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)

css="""
#col-container {max-width: 700px; margin-left: auto; margin-right: auto;}
"""

title = """
<div style="text-align: center;max-width: 700px;">
    <p>Hello Dave, how can I help today?<br />
</div>
"""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
  with gr.Tab("Google|Wikipedia|Arxiv"):
    with gr.Column(elem_id="col-container"):
        gr.HTML(title)
        with gr.Row():
            question = gr.Textbox(label="Question", placeholder="Type your question and hit Enter ")
        chatbot = gr.Chatbot([], elem_id="chatbot")
        chatbot.like(vote, None, None)
        clear = gr.Button("Clear")
    question.submit(add_text, [chatbot, question], [chatbot, question], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch(debug=True)